In [1]:
import argparse, time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph
from dgl.data import register_data_args, load_data
from dgl.data import BitcoinOTC
import datetime

In [ ]:
node_dim = 256

In [2]:
#Trying DGL's implementation of BitcoinOTC class
data = BitcoinOTC()
print("Number of graphs: ")
print(len(data.graphs))
traingr = data.graphs[94]
traingr.number_of_edges()

Number of graphs: 
136


32812

In [12]:
graphs = []

data  = np.loadtxt('soc-sign-bitcoinotc.csv', delimiter=',').astype(np.int64)
data[:, 0:2] = data[:, 0:2] - data[:, 0:2].min()
num_nodes = data[:, 0:2].max() - data[:, 0:2].min() + 1
delta = datetime.timedelta(days=14).total_seconds()
# The source code is not released, but the paper indicates there're
# totally 137 samples. The cutoff below has exactly 137 samples.
time_index = np.around(
    (data[:, 3] - data[:, 3].min())/delta).astype(np.int64)
for i in range(time_index.max()):
    g = DGLGraph()
    g.add_nodes(num_nodes)
    row_mask = time_index <= i
    edges = data[row_mask][:, 0:2]
    rate = data[row_mask][:, 2]
    g.add_edges(edges[:, 0], edges[:, 1])
    g.edata['h'] = rate.reshape(-1, 1)
    g.ndata['h'] = torch.zeros(num_nodes, 256)
    graphs.append(g)

In [4]:
print(data)

[[         5          1          4 1289241911]
 [         5          4          2 1289241941]
 [         0         14          1 1289243140]
 ...
 [      2730       4896          5 1453679434]
 [        12       1127          1 1453679632]
 [      1127         12          2 1453684323]]


In [15]:
graphs[0].ndata['h']

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

## Model definition